In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

c:\Users\alouk\.conda\envs\langchain\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\alouk\.conda\envs\langchain\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\alouk\.conda\envs\langchain\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Found cached dataset yelp_review_full (C:/Users/alouk/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\alouk\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-9afe5ee164297b5d.arrow
Loading cached processed dataset at C:\Users\alouk\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-1345b17724d6e729.arrow


In [3]:
import torch
torch.cuda.empty_cache()
#Remove the text column because the model does not accept raw text as an input:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])


In [4]:
#Rename the label column to labels because the model expects the argument to be named labels:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [5]:
#Set the format of the dataset to return PyTorch tensors instead of lists:
tokenized_datasets.set_format("torch")

In [6]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at C:\Users\alouk\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-c0d91458808f8eb3.arrow
Loading cached shuffled indices for dataset at C:\Users\alouk\.cache\huggingface\datasets\yelp_review_full\yelp_review_full\1.0.0\e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf\cache-862a506804d0efd5.arrow


In [7]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [9]:
import transformers
config = transformers.AutoConfig.from_pretrained(
  'mosaicml/mpt-7b',
  trust_remote_code=True
)
# config.attn_config['attn_impl'] = 'triton'

In [11]:
import pandas as pd
pd.options.display.max_seq_items = 2000
config

MPTConfig {
  "_name_or_path": "mosaicml/mpt-7b",
  "architectures": [
    "MPTForCausalLM"
  ],
  "attn_config": {
    "alibi": true,
    "alibi_bias_max": 8,
    "attn_impl": "torch",
    "attn_pdrop": 0,
    "attn_type": "multihead_attention",
    "attn_uses_sequence_id": false,
    "clip_qkv": null,
    "prefix_lm": false,
    "qk_ln": false,
    "softmax_scale": null
  },
  "auto_map": {
    "AutoConfig": "mosaicml/mpt-7b--configuration_mpt.MPTConfig",
    "AutoModelForCausalLM": "mosaicml/mpt-7b--modeling_mpt.MPTForCausalLM"
  },
  "d_model": 4096,
  "emb_pdrop": 0,
  "embedding_fraction": 1.0,
  "expansion_ratio": 4,
  "init_config": {
    "emb_init_std": null,
    "emb_init_uniform_lim": null,
    "fan_mode": "fan_in",
    "init_div_is_residual": true,
    "init_gain": 0,
    "init_nonlinearity": "relu",
    "init_std": 0.02,
    "name": "kaiming_normal_",
    "verbose": 0
  },
  "init_device": "cpu",
  "learned_pos_emb": true,
  "logit_scale": null,
  "max_seq_len": 2048,
  "m

In [11]:
import gc

gc.collect()

torch.cuda.empty_cache()

In [10]:
import transformers
import torch
torch.cuda.empty_cache()

import transformers
model = transformers.AutoModelForCausalLM.from_pretrained(
  'mosaicml/mpt-7b',
  trust_remote_code=True,
  torch_dtype = torch.bfloat16,
)
model.eval()
model.to("cuda:0")

C:\Users\alouk\.cache\huggingface\modules\transformers_modules\mosaicml\mpt-7b\d8304854d4877849c3c0a78f3469512a84419e84\attention.py:148: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9   torch_dtype = torch.bfloat16,                                                             │
│   10 )                                                                                           │
│   11 model.eval()                                                                                │
│ ❱ 12 model.to("cuda:0")                                                                          │
│   13                                                                                             │
│                                                                                                  │
│ c:\Users\alouk\.conda\envs\langchain\Lib\site-packages\transformers\modeling_utils.py:1878 in to │
│                                                                                                  │
│   1875 │   │   │   │   " model has already been set to the correct devices and casted to the co  │
│   1876 │   │   │   )                                                                             │
│   1877 │   │   else:                                                                             │
│ ❱ 1878 │   │   │   return super().to(*args, **kwargs)                                            │
│   1879 │                                                                                         │
│   1880 │   def half(self, *args):                                                                │
│   1881 │   │   # Checks if the model has been loaded in 8-bit                                    │
│                                                                                                  │
│ C:\Users\alouk\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\module.py:1145 in │
│ to                                                                                               │
│                                                                                                  │
│   1142 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)                │
│   1143 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() else No  │
│   1144 │   │                                                                                     │
│ ❱ 1145 │   │   return self._apply(convert)                                                       │
│   1146 │                                                                                         │
│   1147 │   def register_full_backward_pre_hook(                                                  │
│   1148 │   │   self,                                                                             │
│                                                                                                  │
│ C:\Users\alouk\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\module.py:797 in  │
│ _apply                                                                                           │
│                                                                                                  │
│    794 │                                                                                         │
│    795 │   def _apply(self, fn):                                                                 │
│    796 │   │   for module in self.children():                                                    │
│ ❱  797 │   │   │   module._apply(fn)                                                             │
│    798 │   │                                                                                     │
│    799 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    800 │   │   │   if torch._has_compatible_shallow_copy_ty

In [10]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [11]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

# Define and use tensors
tensor = torch.tensor([1, 2, 3]).cuda()
# Use the tensor

# Clear CUDA memory
del tensor
torch.cuda.empty_cache()


In [13]:
import torch
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 import torch                                                                                 │
│   2 torch.cuda.empty_cache()                                                                     │
│   3 device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")          │
│ ❱ 4 model.to(device)                                                                             │
│   5                                                                                              │
│                                                                                                  │
│ c:\Users\alouk\.conda\envs\langchain\Lib\site-packages\transformers\modeling_utils.py:1878 in to │
│                                                                                                  │
│   1875 │   │   │   │   " model has already been set to the correct devices and casted to the co  │
│   1876 │   │   │   )                                                                             │
│   1877 │   │   else:                                                                             │
│ ❱ 1878 │   │   │   return super().to(*args, **kwargs)                                            │
│   1879 │                                                                                         │
│   1880 │   def half(self, *args):                                                                │
│   1881 │   │   # Checks if the model has been loaded in 8-bit                                    │
│                                                                                                  │
│ C:\Users\alouk\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\module.py:1145 in │
│ to                                                                                               │
│                                                                                                  │
│   1142 │   │   │   │   │   │   │   non_blocking, memory_format=convert_to_format)                │
│   1143 │   │   │   return t.to(device, dtype if t.is_floating_point() or t.is_complex() else No  │
│   1144 │   │                                                                                     │
│ ❱ 1145 │   │   return self._apply(convert)                                                       │
│   1146 │                                                                                         │
│   1147 │   def register_full_backward_pre_hook(                                                  │
│   1148 │   │   self,                                                                             │
│                                                                                                  │
│ C:\Users\alouk\AppData\Roaming\Python\Python311\site-packages\torch\nn\modules\module.py:797 in  │
│ _apply                                                                                           │
│                                                                                                  │
│    794 │                                                                                         │
│    795 │   def _apply(self, fn):                                                                 │
│    796 │   │   for module in self.children():                                                    │
│ ❱  797 │   │   │   module._apply(fn)                                                             │
│    798 │   │                                                                                     │
│    799 │   │   def compute_should_use_set_data(tensor, tensor_applied):                          │
│    800 │   │   │   if torch._has_compatible_shallow_copy_ty